<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b23e93c87062407aeb073a074019f95eb51c75c2787a5c1941e897b000069f44
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-16 11:23:18
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.41 C
-------------------
Today PnL: -15.96 K (-0.11%)
Current PnL: -26.15 L (-16.89%)
CY Booked + Current PnL: -11.86 L (-7.66%)
-------------------
Total profit:  1.46 L
Total loss:  -27.61 L
-------------------
Total Booked + Current PnL: 15.12 L (11.83%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.14%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.79 L (64.43%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.43%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.79,-17.94,21.98,0.09,18176.0,-18083.0,82694.0,91.89,36.0,M-SC,2.34,253.0,-0.99,0.59,8.64,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-0.14,8.50,10.32,19.69,21469.0,16293.0,208035.0,-2.04,57.0,X-MC,2.52,78.0,0.76,1.48,15.87,XY25,NTT,AC
39,INDIGOPNTS,1408.00,-1.02,-12.58,14.41,0.02,21991.0,-21952.0,152607.0,132.70,51.0,M-SC,14.83,234.0,-1.00,1.09,32.74,OX40N,NTT,PAINTS
50,MASFIN,398.61,-1.24,-1.27,23.62,22.05,22849.0,-1245.0,96735.0,-15.09,56.0,H-SC,6.93,164.0,-0.05,0.69,40.61,XR,ATH,FINANCE
43,ITC,452.00,0.47,-0.93,11.83,10.78,28164.0,-2244.0,238074.0,-41.15,50.0,X-LC,1.93,5.0,-0.08,1.70,4.25,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JCHAC,2282.00,2.22,-33.50,50.35,-0.01,42796.0,-42808.0,84997.0,17291.30,40.0,M-SC,7.30,233.0,-1.00,0.61,8.79,OX40N,NTT,AC
23,DMART,5391.45,1.98,-6.22,38.20,29.60,89418.0,-15522.0,234078.0,-20.03,42.0,X-LC,5.64,19.0,-0.17,1.67,14.61,X40N,ATH,FMCG
16,CERA,9475.00,1.96,-30.95,79.42,23.89,96466.0,-54440.0,121463.0,-33.41,36.0,H-SC,10.61,157.0,-0.56,0.87,4.90,OX40N,NTT,CERAMICS
61,ROUTE,2227.21,1.94,-46.93,218.56,69.05,154334.0,-62452.0,70614.0,-57.12,53.0,H-SC,22.86,140.0,-0.40,0.50,8.90,SR,ATH,IT
80,VALIANTORG,838.00,1.46,-43.76,209.68,74.15,192914.0,-71601.0,92004.0,-265.32,47.0,H-SC,22.47,149.0,-0.37,0.66,17.93,XR,NTT,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RAJOOENG,143.10,-2.06,-35.19,115.41,39.61,76667.0,-36070.0,66430.0,-56.22,35.0,H-SC,28.71,135.0,-0.47,0.47,4.66,AR,ATH,MISC
32,HCLTECH,1922.01,-1.60,7.54,15.99,24.73,41600.0,18241.0,260165.0,12.34,58.0,X-LC,6.85,7.0,0.44,1.86,23.92,X40,ATH,IT
45,JIOFIN,387.00,-1.49,-5.06,31.12,24.48,78992.0,-13536.0,253829.0,-6.69,42.0,X-LC,5.59,36.0,-0.17,1.81,47.13,XY24,BTT,FINANCE
58,RECLTD,446.00,-1.40,-8.58,32.07,20.74,59565.0,-17435.0,185735.0,21.78,30.0,M-MC,7.68,189.0,-0.29,1.32,0.00,XY25,NTT,FINANCE
79,VAIBHAVGBL,521.00,-1.36,-26.40,118.42,60.77,159312.0,-48244.0,134531.0,55.37,48.0,H-SC,4.46,159.0,-0.30,0.96,21.16,XR,NTT,JEWELLERY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.84,-3.16,114.69,107.91,163984.0,-4664.0,142980.0,-21.36,54.0,M-SC,12.61,216.0,-0.03,1.02,6.61,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.02,-12.58,14.41,0.02,21991.0,-21952.0,152607.0,132.70,51.0,M-SC,14.83,234.0,-1.00,1.09,32.74,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.79,-17.94,21.98,0.09,18176.0,-18083.0,82694.0,91.89,36.0,M-SC,2.34,253.0,-0.99,0.59,8.64,OX40N,NTT,DURABLES
66,SIS,528.00,0.14,-24.12,59.81,21.26,50391.0,-26780.0,84252.0,1987.56,50.0,H-SC,5.12,166.0,-0.53,0.60,13.83,OX40N,NTT,MISC
47,KANSAINER,340.00,0.04,-24.42,50.14,13.47,102188.0,-65862.0,203805.0,-68.90,38.0,H-SC,5.17,158.0,-0.64,1.45,3.63,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.59,6.72,61.41,72.27,111053.0,11391.0,180839.0,-8.35,65.0,M-LC,2.46,99.0,0.1,1.29,13.31,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.59,6.72,61.41,72.27,111053.0,11391.0,180839.0,-8.35,65.0,M-LC,2.46,99.0,0.10,1.29,13.31,XR,NTT,IT
38,INDIAMART,4810.62,-0.01,-2.62,112.28,106.72,134858.0,-3227.0,120109.0,-51.77,39.0,H-SC,2.95,139.0,-0.02,0.86,20.15,AR,ATH,MISC
25,FINCABLES,1641.55,0.84,-3.16,114.69,107.91,163984.0,-4664.0,142980.0,-21.36,54.0,M-SC,12.61,216.0,-0.03,1.02,6.61,OX40N,ATH,CABLES
50,MASFIN,398.61,-1.24,-1.27,23.62,22.05,22849.0,-1245.0,96735.0,-15.09,56.0,H-SC,6.93,164.0,-0.05,0.69,40.61,XR,ATH,FINANCE
1,ABB,7934.00,-0.38,-3.40,50.69,45.57,128104.0,-8899.0,252720.0,-37.87,58.0,H-MC,4.55,120.0,-0.07,1.80,7.53,AR,NTT,ELECTRICAL


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.35,-18.24,90.15,55.47,179325.0,-44368.0,198918.0,-27.60,25.0,X-MC,12.93,64.0,-0.25,1.42,1.92,X40N,ATH,FINANCE
53,PAGEIND,51605.07,-0.50,-9.25,41.17,28.11,75248.0,-18635.0,182775.0,-34.25,29.0,X-MC,12.92,55.0,-0.25,1.30,0.00,X40,ATH,APPARELS
75,TMPV,600.00,-0.29,-29.05,73.34,22.98,117540.0,-65631.0,160267.0,-26.74,30.0,X-LC,6.79,3.0,-0.56,1.14,0.80,XY24,NTT,AUTO
51,MEDANTA,1486.00,-1.13,-5.21,30.90,24.08,37182.0,-6619.0,120331.0,-12.87,31.0,X-SC,5.13,89.0,-0.18,0.86,13.38,XY24,NTT,HEALTHCARE
11,BATAINDIA,2096.00,-0.92,-38.69,120.52,35.20,95081.0,-49778.0,78892.0,3.39,32.0,X-SC,16.12,92.0,-0.52,0.56,0.21,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.33,-1.38,26.21,24.48,65103.0,-3465.0,248391.0,-81.09,43.0,X-SC,0.32,86.0,-0.05,1.77,21.44,X40N,NTT,MISC
43,ITC,452.0,0.47,-0.93,11.83,10.78,28164.0,-2244.0,238074.0,-41.15,50.0,X-LC,1.93,5.0,-0.08,1.70,4.25,X40,NTT,FMCG
33,HINDUNILVR,2922.0,0.09,-10.70,27.29,13.67,62644.0,-27513.0,229550.0,-17.44,34.0,X-LC,2.13,14.0,-0.44,1.64,8.37,XY25,NTT,FMCG
36,ICICIPRULI,790.0,-0.05,7.26,21.88,30.73,46235.0,14308.0,211313.0,-17.82,64.0,X-MC,2.32,75.0,0.31,1.51,20.96,X40,ATH,INSURANCE
78,UNITDSPR,1644.0,0.35,6.32,13.58,20.77,33808.0,14809.0,248953.0,-2.93,57.0,X-MC,2.44,62.0,0.44,1.78,13.68,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.07,-0.50,-9.25,41.17,28.11,75248.0,-18635.0,182775.0,-34.25,29.0,X-MC,12.92,55.0,-0.25,1.30,0.00,X40,ATH,APPARELS
3,ACC,3906.00,-0.26,-25.49,120.30,64.15,213292.0,-60651.0,177300.0,-56.31,34.0,X-SC,2.59,82.0,-0.28,1.26,0.07,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-0.92,-38.69,120.52,35.20,95081.0,-49778.0,78892.0,3.39,32.0,X-SC,16.12,92.0,-0.52,0.56,0.21,X40,NTT,FOOTWEAR
75,TMPV,600.00,-0.29,-29.05,73.34,22.98,117540.0,-65631.0,160267.0,-26.74,30.0,X-LC,6.79,3.0,-0.56,1.14,0.80,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-0.35,-18.24,90.15,55.47,179325.0,-44368.0,198918.0,-27.60,25.0,X-MC,12.93,64.0,-0.25,1.42,1.92,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.40,-29.27,101.96,42.84,46878.0,-19029.0,45977.0,-53.72,44.0,X-SC,35.52,83.0,-0.41,0.33,4.02,XY24,NTT,MISC
59,RELAXO,1176.00,0.20,-47.54,190.33,52.31,144934.0,-69011.0,76149.0,-44.15,43.0,X-SC,6.90,91.0,-0.48,0.54,1.93,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.92,-38.69,120.52,35.20,95081.0,-49778.0,78892.0,3.39,32.0,X-SC,16.12,92.0,-0.52,0.56,0.21,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-1.13,-5.21,30.90,24.08,37182.0,-6619.0,120331.0,-12.87,31.0,X-SC,5.13,89.0,-0.18,0.86,13.38,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.03,-16.36,71.36,43.32,97207.0,-26652.0,136220.0,-28.18,38.0,X-SC,4.74,90.0,-0.27,0.97,4.59,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.87,-11.95,37.09,20.71,123520.0,-45208.0,333029.0,-24.27,58.0,X-LC,4.46,1.0,-0.37,2.37,11.28,X40,ATH,IT
41,INFY,1972.00,0.54,9.38,22.73,34.24,77793.0,29360.0,342248.0,-14.93,64.0,X-LC,3.59,2.0,0.38,2.44,18.44,X40,NTT,IT
75,TMPV,600.00,-0.29,-29.05,73.34,22.98,117540.0,-65631.0,160267.0,-26.74,30.0,X-LC,6.79,3.0,-0.56,1.14,0.80,XY24,NTT,AUTO
81,VBL,671.64,1.02,-2.55,39.21,35.67,120690.0,-8039.0,307803.0,-14.21,60.0,X-LC,4.11,4.0,-0.07,2.19,10.96,X40N,ATH,FMCG
43,ITC,452.00,0.47,-0.93,11.83,10.78,28164.0,-2244.0,238074.0,-41.15,50.0,X-LC,1.93,5.0,-0.08,1.70,4.25,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.79,-37.65,113.14,32.89,54461.0,-29070.0,48136.0,-694.37,53.0,L-MC,5.96,259.0,-0.53,0.34,32.56,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.73,-19.08,124.52,81.67,94522.0,-17901.0,75909.0,6816.67,38.0,L-SC,12.22,271.0,-0.19,0.54,47.00,XR,NTT,CERAMICS
50,MASFIN,398.61,-1.24,-1.27,23.62,22.05,22849.0,-1245.0,96735.0,-15.09,56.0,H-SC,6.93,164.0,-0.05,0.69,40.61,XR,ATH,FINANCE
13,BSOFT,831.70,-0.45,-21.53,92.17,50.79,101722.0,-30289.0,110364.0,0.50,67.0,H-SC,6.38,151.0,-0.30,0.79,28.50,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.02,-12.58,14.41,0.02,21991.0,-21952.0,152607.0,132.70,51.0,M-SC,14.83,234.0,-1.00,1.09,32.74,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-0.45,-21.53,92.17,50.79,101722.0,-30289.0,110364.0,0.50,67.0,H-SC,6.38,151.0,-0.30,0.79,28.50,XR,ATH,IT
84,WIPRO,420.00,-0.59,6.72,61.41,72.27,111053.0,11391.0,180839.0,-8.35,65.0,M-LC,2.46,99.0,0.10,1.29,13.31,XR,NTT,IT
41,INFY,1972.00,0.54,9.38,22.73,34.24,77793.0,29360.0,342248.0,-14.93,64.0,X-LC,3.59,2.0,0.38,2.44,18.44,X40,NTT,IT
81,VBL,671.64,1.02,-2.55,39.21,35.67,120690.0,-8039.0,307803.0,-14.21,60.0,X-LC,4.11,4.0,-0.07,2.19,10.96,X40N,ATH,FMCG
1,ABB,7934.00,-0.38,-3.40,50.69,45.57,128104.0,-8899.0,252720.0,-37.87,58.0,H-MC,4.55,120.0,-0.07,1.80,7.53,AR,NTT,ELECTRICAL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.26
1,25,44.91
2,50,76.65


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.96
MC    30.95
LC    24.08
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.71
X40      22.98
X40N     13.07
XR        9.77
XY25      9.08
AR        9.05
OX40N     7.57
SR        0.99
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.48
H-SC    23.83
X-LC    20.67
M-SC    11.69
X-SC     8.03
H-MC     4.81
L-SC     1.41
M-MC     1.32
M-LC     1.29
H-LC     1.12
L-LC     1.00
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.47,-6.57,40.83
IT,12.99,-15.36,73.36
FINANCE,9.47,-19.95,72.12
MISC,7.27,-27.46,79.60
ELECTRICAL,6.01,-11.13,51.84
PAINTS,5.86,-12.31,28.99
INSURANCE,4.79,-0.96,35.51
PHARMA,4.11,-1.95,34.04
AUTO,2.81,-33.31,79.53


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3175487.0,21
AR,1301576.0,10
XR,1292641.0,13
X40,1000509.0,14
X40N,852031.0,9
OX40N,731330.0,10
XY25,367464.0,6
SR,280863.0,2
MH,76580.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3637627.0,25
M-SC,1421194.0,15
X-MC,1393677.0,16
X-LC,827457.0,11
X-SC,792394.0,8
H-MC,403818.0,3
L-SC,262731.0,3
M-LC,111053.0,1
H-LC,75869.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1271408.0      6
           AR         891224.0      5
M-SC       XY24       817416.0      6
H-SC       XR         775895.0      7
X-MC       X40        483911.0      7
           XY24       397840.0      3
H-SC       OX40N      341657.0      4
X-MC       X40N       334679.0      4
X-LC       X40        328800.0      5
X-SC       X40N       307244.0      3
M-SC       OX40N      304597.0      5
X-SC       XY24       297352.0      3
H-SC       SR         280863.0      2
X-LC       X40N       210108.0      2
H-MC       AR         208879.0      2
X-LC       XY24       196532.0      2
H-MC       XY24       194939.0      1
X-SC       X40        187798.0      2
L-SC       XR         177655.0      2
X-MC       XY25       177247.0      2
M-SC       XR         173577.0      2
           AR         125604.0      2
M-LC       XR         111053.0      1
X-LC       XY25        92017.0      2
L-SC       OX40N       85076.0      1
H-SC       MH          76580.0      1
H-LC       AR          75869.0      1
M-MC       XY25        59565.0      1
L-MC       XR          54461.0      1
L-LC       XY25        38635.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
